In [1]:
import numpy as np
import math

In [110]:
#red in the file and parse it
def read_input_file(file):
    with open(file, 'r') as f:
        # Read deltax, deltay, dx, dy, sigma2
        row1 = f.readline().strip().split()
        deltax = float(row1[0])
        deltay = float(row1[1])
        dx = float(row1[2])
        dy = float(row1[3])
        sigma2 = float(row1[4])
        
        # Read M and N
        row2 = f.readline().strip().split()
        M = int(row2[0])
        N = int(row2[1])
        # read a
        a = []
        for i in range(M):
            row = f.readline().strip().split()
            a.append([float(val) for val in row])
        # read m,n
        row = f.readline().strip().split()
        m = int(row[0])
        n = int(row[1])
        #read t
        t = []
        for i in range(m):
            row = f.readline().strip().split()
            t.append([float(val) for val in row])

        return deltax, deltay, dx, dy, sigma2, M, N, a, m, n, t

deltax, deltay, dx, dy, sigma2, M, N, a, m, n, t = read_input_file('PS2 test 2.txt')
print(deltax, deltay, dx, dy, sigma2, M, N, a, m, n, t)

0.1 0.1 0.01 0.01 5.0 10 10 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0], [0.0, 0.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0], [0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]] 5 5 [[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.5, 1.0, 0.5, 0.0], [0.0, 1.0, 1.0, 1.0, 0.0], [0.0, 0.5, 1.0, 0.5, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [89]:
#objective function f(z,t,x,y)
def obj(a,t,x,y):
    sum=0
    for i in range(m):
        for j in range(n):
            intense=intensity(a,i+x,j+y)
            sum+=((t[i][j]-intense)*(t[i][j]-intense))
    return sum

#compute the intensity value
def intensity(a,x,y):
    denom=0
    num=0
    for p in range(M):
        for q in range(N):
            exponent = math.exp(-1*sigmasq*((x-p)*(x-p)+(y-q)*(y-q)))
            num+=a[p][q]*exponent
            denom+=exponent
    if denom==0:
        return 0
    else:
        return float(num)/float(denom)

In [83]:
#once we have best i and j, we can do newton raphson on that position
def newton_raphson(a,t,best_i,best_j):
    #make best_i and best_j into a vector
    best = np.array([best_i,best_j])
    for i in range(10):
        hes =hessian(a,t,best)
        grad = gradient(a,t,best)
        new_i = (hes[1][1]*grad[0]-hes[0][1]*grad[1])/(hes[0][0]*hes[1][1]-hes[0][1]*hes[0][1])
        new_j = (hes[0][0]*grad[1]-hes[1][0]*grad[0])/(hes[0][0]*hes[1][1]-hes[0][1]*hes[0][1])
        best_i-=new_i
        best_j-=new_j
        if best_i<0 or best_i>5 or best_j<0 or best_j>5:
            return (-1.0,-1.0)
    # check if the value is within the range of the matrix (M-m,N-n)
    # if not, return -1,-1
    #otherwise, return that position
    return (best_i,best_j)

In [43]:
# compute the hessian
def hessian(a,t,best):
    x=best[0]
    y=best[1]
    hes00 = (obj(a,t,x+dx,y)+obj(a,t,x-dx,y)-2*obj(a,t,x,y))/((dx)*(dx))
    hes01 = (obj(a,t,x+dx,y+dy)-obj(a,t,x+dx,y-dy)-obj(a,t,x-dx,y+dy)+obj(a,t,x-dx,y-dy))/(4*dx*dy)
    hes10 = (obj(a,t,x+dx,y+dy)-obj(a,t,x+dx,y-dy)-obj(a,t,x-dx,y+dy)+obj(a,t,x-dx,y-dy))/(4*dx*dy)
    hes11 = (obj(a,t,x,y+dy)+obj(a,t,x,y-dy)-2*obj(a,t,x,y))/((dy)*(dy))
    ans = np.array([[hes00,hes01],[hes10,hes11]])
    return ans

    

In [41]:
#compute the gradient
def gradient(a,t,best):
    x=best[0]
    y=best[1]
    x = fprime_x(a,t,x,y)
    y = fprime_y(a,t,x,y)
    ans = np.array([x,y])
    return ans

#compute the gradient for x
def fprime_x(a,t,x,y):
    #calls the objective on f(x+dx)-f(x-dx)/2dx
    return (obj(a,t,x+dx,y)-obj(a,t,x-dx,y))/(2*dx)

#compute the gradient for y
def fprime_y(a,t,x,y):
    #calls the objective on f(y+dy)-f(y-dy)/2dy
    return (obj(a,t,x,y+dy)-obj(a,t,x,y-dy))/(2*dy)   

In [92]:
#test case 2

# loop through all starting points and do newton raphson and store the best soln in a list
obj_vals = np.zeros(0)
i_vals = np.zeros(0)
j_vals = np.zeros(0)
for i in np.arange(0,M-m,deltax):
    for j in np.arange(0,N-n,deltay):  

        # do newton raphson and add the result to the list of the objective function of the end result
        print(i,j)
        best_x,best_y = newton_raphson(a,t,i,j)
        obj_val = obj(a,t,best_x,best_y)
        obj_vals=np.append(obj_vals,obj_val)
        i_vals=np.append(i_vals,best_x)
        j_vals=np.append(j_vals,best_y)

#take min of list
best_obj_index = np.argmin(obj_vals)
best_i = i_vals[best_obj_index]
best_j=j_vals[best_obj_index]
#return the position and x and y of the min 
print(best_i,best_j)

0.0 0.0


0.0 0.1
0.0 0.2
0.0 0.30000000000000004
0.0 0.4
0.0 0.5
0.0 0.6000000000000001
0.0 0.7000000000000001
0.0 0.8
0.0 0.9
0.0 1.0
0.0 1.1
0.0 1.2000000000000002
0.0 1.3
0.0 1.4000000000000001
0.0 1.5
0.0 1.6
0.0 1.7000000000000002
0.0 1.8
0.0 1.9000000000000001
0.0 2.0
0.0 2.1
0.0 2.2
0.0 2.3000000000000003
0.0 2.4000000000000004
0.0 2.5
0.0 2.6
0.0 2.7
0.0 2.8000000000000003
0.0 2.9000000000000004
0.0 3.0
0.0 3.1
0.0 3.2
0.0 3.3000000000000003
0.0 3.4000000000000004
0.0 3.5
0.0 3.6
0.0 3.7
0.0 3.8000000000000003
0.0 3.9000000000000004
0.0 4.0
0.0 4.1000000000000005
0.0 4.2
0.0 4.3
0.0 4.4
0.0 4.5
0.0 4.6000000000000005
0.0 4.7
0.0 4.800000000000001
0.0 4.9
0.1 0.0
0.1 0.1
0.1 0.2
0.1 0.30000000000000004
0.1 0.4
0.1 0.5
0.1 0.6000000000000001
0.1 0.7000000000000001
0.1 0.8
0.1 0.9
0.1 1.0
0.1 1.1
0.1 1.2000000000000002
0.1 1.3
0.1 1.4000000000000001
0.1 1.5
0.1 1.6
0.1 1.7000000000000002
0.1 1.8
0.1 1.9000000000000001
0.1 2.0
0.1 2.1
0.1 2.2
0.1 2.3000000000000003
0.1 2.4000000000000004
0.

In [94]:
#test case 1


file_path = 'PS2 test 1.txt'
deltax, deltay, dx, dy, sigma2, M, N, a, m, n, t = read_input_file(file_path)
deltax = deltax*10
deltay = deltay *10

# loop through all starting points and do newton raphson and store the best soln in a list
obj_vals = np.zeros(0)
i_vals = np.zeros(0)
j_vals = np.zeros(0)
for i in np.arange(0,M-m,deltax):
    for j in np.arange(0,N-n,deltay):  

        # do newton raphson and add the result to the list of the objective function of the end result
        print(i,j)
        best_x,best_y = newton_raphson(a,t,i,j)
        obj_val = obj(a,t,best_x,best_y)
        obj_vals=np.append(obj_vals,obj_val)
        i_vals=np.append(i_vals,best_x)
        j_vals=np.append(j_vals,best_y)

#take min of list
best_obj_index = np.argmin(obj_vals)
best_i = i_vals[best_obj_index]
best_j=j_vals[best_obj_index]
#return the position and x and y of the min 
print(best_i,best_j)

0.0 0.0
0.0 0.1
0.0 0.2
0.0 0.30000000000000004
0.0 0.4
0.0 0.5
0.0 0.6000000000000001
0.0 0.7000000000000001
0.0 0.8
0.0 0.9
0.0 1.0
0.0 1.1
0.0 1.2000000000000002
0.0 1.3
0.0 1.4000000000000001
0.0 1.5
0.0 1.6
0.0 1.7000000000000002
0.0 1.8
0.0 1.9000000000000001
0.0 2.0
0.0 2.1
0.0 2.2
0.0 2.3000000000000003
0.0 2.4000000000000004
0.0 2.5
0.0 2.6
0.0 2.7
0.0 2.8000000000000003
0.0 2.9000000000000004
0.0 3.0
0.0 3.1
0.0 3.2
0.0 3.3000000000000003
0.0 3.4000000000000004
0.0 3.5
0.0 3.6
0.0 3.7
0.0 3.8000000000000003
0.0 3.9000000000000004
0.0 4.0
0.0 4.1000000000000005
0.0 4.2
0.0 4.3
0.0 4.4
0.0 4.5
0.0 4.6000000000000005
0.0 4.7
0.0 4.800000000000001
0.0 4.9
0.1 0.0
0.1 0.1
0.1 0.2
0.1 0.30000000000000004
0.1 0.4
0.1 0.5
0.1 0.6000000000000001
0.1 0.7000000000000001
0.1 0.8
0.1 0.9
0.1 1.0
0.1 1.1
0.1 1.2000000000000002
0.1 1.3
0.1 1.4000000000000001
0.1 1.5
0.1 1.6
0.1 1.7000000000000002
0.1 1.8
0.1 1.9000000000000001
0.1 2.0
0.1 2.1
0.1 2.2
0.1 2.3000000000000003
0.1 2.40000000000

#pseudocode
best_obj = []
best_xs = []
best_y = []
#range through every point on the grid
for x in every M-m by deltax:
    for y in every N-n by deltay:
        #do newton raphson of that point
        for k in range 10:
            # update x and y using newton raphson method
            (x,y)=(x,y)-J(x,y)^(-1)gradient(x,y)
        #add optimized point to the list of possible solutions
        best_obj.append(f(x,y))
        best_xs.append(x)
        best_ys.append(y)
#find the best overall value from list of solutions from every point
best_index = argmax(best_obj)
best_x = best_xs[best_index]
best_y = best_ys[best_index]

return best_x,best_y